In [1]:
import pandas as pd
import datetime as dt
import os

In [2]:
from mysql.connector import (connection)

In [3]:
from mysql.connector import Error


# Get Tweets

In [4]:
# fetch values from environment variables and set the target database
hostname = os.environ['MYSQL_HOST']
username = os.environ['MYSQL_USER']
password = os.environ['MYSQL_PASSWORD']
dbname = 'found'

# establish connection to db1 database in your mysql service
cnx = connection.MySQLConnection(user=username,
                                 password=password,
                                 host=hostname,
                                 database=dbname)


In [5]:
# DEFINE QUERY
# UPDATE DATE
query =("SELECT * FROM Foundation_Tweets INNER JOIN Foundations ON Foundation_Tweets.Foundation_Name = Foundations.Foundation_Name WHERE Tweeted_by_Foundation = 'True' AND Tweet_date > '2023-02-22'")

In [6]:
df = pd.read_sql_query(query, cnx)

In [7]:
df.shape

(2744, 9)

In [8]:
# remove duplicated foundation name column
df = df.loc[:,~df.columns.duplicated()]

# clean region column
df["Region"] = df["Region"].str.replace("Global ", "Global")

# drop duplicate tweets
df = df.drop_duplicates(subset='Tweet', keep="first")

In [9]:
df.shape

(2660, 8)

In [10]:
df.columns = df.columns.str.lower()

In [11]:
df.head()

,id,foundation_name,tweet,tweet_date,tweeted_by_foundation,query_term,handle_type,region
0,1628567768756031490,"George Lucas Family Foundation (CA), USA,",RT @edutopia: In this escape-room-inspired loc...,2023-02-23,True,@edutopia,Foundation,None
1,1628776210267271170,"The Pew Charitable Trust, USA,",RT @pewtrusts: These 4 stories of people from ...,2023-02-23,True,@pewtrusts,Foundation,None
2,1628799303123628038,"John D. and Catherine T. MacArthur Foundation,...",RT @KathyIm2: I'll be talking about the ethica...,2023-02-23,True,@macfound,Foundation,None
3,1628774265410531334,"John D. and Catherine T. MacArthur Foundation,...",RT @degarciaknight: “Now is the time to respon...,2023-02-23,True,@macfound,Foundation,None
4,1628851214254280704,"Rockefeller Foundation, USA,",Funded by @RockefellerFdn &amp; @gatesfoundati...,2023-02-23,True,@RockefellerFdn,Foundation,None


## Run Model

In [12]:
import numpy as np
import re
from tner import TransformersNER

2023-03-21 18:59:59.564766: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/oracle/instantclient_12_1:/usr/lib/jvm/java-8-openjdk-amd64/jre/lib/amd64/server:
2023-03-21 18:59:59.564809: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [13]:
model = TransformersNER("tner/bert-large-tweetner7-2020")

2023-03-21 19:00:08 INFO     initialize language model with `tner/bert-large-tweetner7-2020`


Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

2023-03-21 19:00:36 INFO     use CRF
2023-03-21 19:00:36 INFO     loading pre-trained CRF layer
2023-03-21 19:00:36 INFO     label2id: {'B-corporation': 0, 'B-creative_work': 1, 'B-event': 2, 'B-group': 3, 'B-location': 4, 'B-person': 5, 'B-product': 6, 'I-corporation': 7, 'I-creative_work': 8, 'I-event': 9, 'I-group': 10, 'I-location': 11, 'I-person': 12, 'I-product': 13, 'O': 14}
2023-03-21 19:00:36 INFO     device   : cpu
2023-03-21 19:00:36 INFO     gpus     : 0


Downloading:   0%|          | 0.00/445 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/653k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [14]:
sample_text = "#Balochistan: Balochistan Liberation Front claims to have targeted &amp; seriously injured two alleged members of “death squad” in an IED attack on 3 February in Pelar area of Jhao."

In [ ]:
model.predict(sample_text)

2023-03-21 19:01:46 INFO     encode all the data: 181
  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
def create_dict(i, model_dict):
    results_dict =  {
    'tagged_text_loci': i,
    'prediction' : model_dict['prediction'][0][i],
    'probability' : model_dict['probability'][0][i],
    'input' : model_dict['input'][0][i]}
    return results_dict

In [21]:
tner_records = []
for row_dict in df.to_dict(orient="records"):
    for ner_tag_dict in [model.predict([row_dict["tweet"]])]:
        for i in range(len(ner_tag_dict['prediction'][0])):
                results_dict = create_dict(i, ner_tag_dict)
                tner_records.append({**row_dict, **results_dict})

2023-02-23 17:48:55 INFO     encode all the data: 1
100%|██████████| 1/1 [00:00<00:00,  2.89it/s]
2023-02-23 17:48:56 INFO     encode all the data: 1
100%|██████████| 1/1 [00:00<00:00,  3.49it/s]
2023-02-23 17:48:56 INFO     encode all the data: 1
100%|██████████| 1/1 [00:00<00:00,  3.23it/s]
2023-02-23 17:48:56 INFO     encode all the data: 1
100%|██████████| 1/1 [00:00<00:00,  3.10it/s]
2023-02-23 17:48:56 INFO     encode all the data: 1
100%|██████████| 1/1 [00:00<00:00,  3.02it/s]
2023-02-23 17:48:57 INFO     encode all the data: 1
100%|██████████| 1/1 [00:00<00:00,  3.41it/s]
2023-02-23 17:48:57 INFO     encode all the data: 1
100%|██████████| 1/1 [00:00<00:00,  3.23it/s]
2023-02-23 17:48:57 INFO     encode all the data: 1
100%|██████████| 1/1 [00:00<00:00,  3.46it/s]
2023-02-23 17:48:58 INFO     encode all the data: 1
100%|██████████| 1/1 [00:00<00:00,  3.34it/s]
2023-02-23 17:48:58 INFO     encode all the data: 1
100%|██████████| 1/1 [00:00<00:00,  3.16it/s]
2023-02-23 17:48:58 

In [22]:
df = pd.DataFrame(tner_records)

In [23]:
df.shape

(427913, 12)

In [24]:
df.to_csv("foundations_tweets_tner_output.csv", index = False)

In [25]:
df.head()

,id,foundation_name,tweet,tweet_date,tweeted_by_foundation,query_term,handle_type,region,tagged_text_loci,prediction,probability,input
0,1401904944404443138,"Hartford Foundation for Public Giving (CT), USA,","RT @BizofGive: ""We think it’s important to par...",2021-06-07,True,@HartfordFdn,Foundation,None,0,B-person,0.860420,RT
1,1401904944404443138,"Hartford Foundation for Public Giving (CT), USA,","RT @BizofGive: ""We think it’s important to par...",2021-06-07,True,@HartfordFdn,Foundation,None,1,O,0.999998,@BizofGive:
2,1401904944404443138,"Hartford Foundation for Public Giving (CT), USA,","RT @BizofGive: ""We think it’s important to par...",2021-06-07,True,@HartfordFdn,Foundation,None,2,O,0.999998,"""We"
3,1401904944404443138,"Hartford Foundation for Public Giving (CT), USA,","RT @BizofGive: ""We think it’s important to par...",2021-06-07,True,@HartfordFdn,Foundation,None,3,O,0.999997,think
4,1401904944404443138,"Hartford Foundation for Public Giving (CT), USA,","RT @BizofGive: ""We think it’s important to par...",2021-06-07,True,@HartfordFdn,Foundation,None,4,O,0.999997,it’s


In [29]:
stop_words = ["i","me","my","myself","we","our","ours","ourselves","you","your","yours","yourself","yourselves","he","him","his","himself","she","her","hers","herself","it","its","itself","they","them","their","theirs","themselves","what","which","who","whom","this","that","these","those","am","is","are","was","were","be","been","being","have","has","had","having","do","does","did","doing","a","an","the","and","but","if","or","because","as","until","while","of","at","by","for","with","about","against","between","into","through","during","before","after","above","below","to","from","up","down","in","out","on","off","over","under","again","further","then","once","here","there","when","where","why","how","all","any","both","each","few","more","most","other","some","such","no","nor","not","only","own","same","so","than","too","very","s","t","can","will","just","don","should","now"]

def create_ner_tag_df(records_list, model_type):
    df = pd.DataFrame(records_list)
    df = df[df['prediction'] != 'O']
    df['sentence_number'] = 1
    df['ner_model'] = model_type
    df.rename(columns = {'id' :'document_id',
                        'tweet': 'sentence_text',
                        'input': 'tagged_text', 
                        'prediction': 'ner_tag'}, inplace = True)
    df = df[['document_id', 'sentence_number', 'sentence_text', 'tagged_text', 'ner_tag', 'tagged_text_loci', 'ner_model', 'probability']]
    df = df[~df['tagged_text'].isin(stop_words)]
    df['document_id'] = df['document_id'].astype(str)
    df['tagged_text_loci'] = df['tagged_text_loci'].astype(str)
    return df

In [30]:
tner_df = create_ner_tag_df(tner_records, "tner/bert-large-tweetner7-2020")

In [31]:
tner_df

,document_id,sentence_number,sentence_text,tagged_text,ner_tag,tagged_text_loci,ner_model,probability
0,1401904944404443138,1,"RT @BizofGive: ""We think it’s important to par...",RT,B-person,0,tner/bert-large-tweetner7-2020,0.860420
7,1401904944404443138,1,"RT @BizofGive: ""We think it’s important to par...",partner,B-event,7,tner/bert-large-tweetner7-2020,0.956221
24,1401904838833737733,1,RT @UConnHDI: UConn Health Disparities Institu...,UConn,B-corporation,2,tner/bert-large-tweetner7-2020,0.999975
25,1401904838833737733,1,RT @UConnHDI: UConn Health Disparities Institu...,Health,I-corporation,3,tner/bert-large-tweetner7-2020,0.999991
26,1401904838833737733,1,RT @UConnHDI: UConn Health Disparities Institu...,Disparities,I-corporation,4,tner/bert-large-tweetner7-2020,0.999992
...,...,...,...,...,...,...,...,...
427883,1628417506103664640,1,"RT @BKpub: Congratulations to Bob Johansen, Jo...",Christine,B-person,9,tner/bert-large-tweetner7-2020,0.999992
427884,1628417506103664640,1,"RT @BKpub: Congratulations to Bob Johansen, Jo...",Bullen;,I-person,10,tner/bert-large-tweetner7-2020,0.999992
427887,1628417506103664640,1,"RT @BKpub: Congratulations to Bob Johansen, Jo...",Office,B-creative_work,13,tner/bert-large-tweetner7-2020,0.999975
427888,1628417506103664640,1,"RT @BKpub: Congratulations to Bob Johansen, Jo...",Shock,I-creative_work,14,tner/bert-large-tweetner7-2020,0.999961


In [32]:
tner_df.to_csv("foundations_tweets_ner_records.csv", index= False)

## Write to Table

In [5]:
tner_df = pd.read_csv("foundations_tweets_ner_records.csv")

In [21]:
# remove RT from tagged text
tner_df = tner_df[tner_df['tagged_text'] != 'RT']

In [22]:
len(tner_df['document_id'].unique())

13880

In [23]:
columns = list(tner_df.columns)
column_str = f"({', '.join(columns)})"
insert_placeholder = ", ".join(["%s" for _ in column_str.split(",")])
ner_val_dict = tner_df.to_dict("records")
save_data = [tuple(row.values()) for row in ner_val_dict]
ner_query = f"INSERT INTO ner_tagged_grants {column_str} VALUES ({insert_placeholder});"

In [28]:
sql.run_insert_query(ner_query, save_data)

NameError: name 'sql' is not defined